In [5]:
%matplotlib inline

import pandas as pd
import numpy as np

from modelclass import model


In [7]:
fmodel      = '''
frml <> a = c(-1) + b $ 
frml <> d1 = x + 3 * e(-1)+ c **2 +a  $ 
frml <> d3 = x + 3 * a(-1)+c **3 $  
Frml <> x = 0.5 * c $'''
mmodel = model(fmodel,modelname = 'My first model')
df = pd.DataFrame({'B': [1,1,1,1],'C':[1,2,3,4],'E':[4,4,4,4]},index=[2018,2019,2020,2021])
base = mmodel(df)
base

Will start calculating: My first model
2019  solved
2020  solved
2021  solved
My first model calculated 


,B,C,E,D1,D3,A,X
2018,1.0,1.0,4.0,0.0,0.0,0.0,0.0
2019,1.0,2.0,4.0,19.0,9.0,2.0,1.0
2020,1.0,3.0,4.0,25.5,34.5,3.0,1.5
2021,1.0,4.0,4.0,34.0,75.0,4.0,2.0


In [8]:
from modelnewton import newton_diff
newton = newton_diff(mmodel)
newton2 = newton_diff(mmodel,forcenum=1)

In [10]:
print(newton.diff_model.equations)

FRML  <> D1__p__A___lag___0 = 1   $
FRML  <> D1__p__X___lag___0 = 1   $
FRML  <> D3__p__A___lag___1 = 3   $
FRML  <> D3__p__X___lag___0 = 1   $


In [11]:
print(newton2.diff_model.equations)

FRML  <> D1__p__A___lag___0 = ((X+3*E(-1)+C**2+(A+0.0025))-(X+3*E(-1)+C**2+(A-0.0025)))/0.005   $
FRML  <> D1__p__X___lag___0 = (((X+0.0025)+3*E(-1)+C**2+A)-((X-0.0025)+3*E(-1)+C**2+A))/0.005   $
FRML  <> D3__p__A___lag___1 = ((X+3*(A(-1)+0.0025)+C**3)-(X+3*(A(-1)-0.0025)+C**3))/0.005   $
FRML  <> D3__p__X___lag___0 = (((X+0.0025)+3*A(-1)+C**3)-((X-0.0025)+3*A(-1)+C**3))/0.005   $


In [12]:
stacked_df = newton2.get_diff_df()
stacked_df.style.format('{0:,.3f}')

In [ ]:
for l,m in matdir.items():
    print(f'Lag:{l} \n {m} \n')

for l,m in matdir.items():
    endomat = m.loc[:,[c for c in m.columns if c in mmodel.endogene]]
    print(f'Lag:{l} \n {endomat} \n')

In [ ]:
fam   = '''
frml <> y = c+i $ 
frml <I> c = pa * y(-1) $
frml <> i = pc * (c-c(-1)) + im$'''

mma = mc.model(fam,modelname = 'Accelerator multiplicator model')
fhma = mp.find_hist_model(fam)  # Find the model to generate lagged values of c
mhma = mc.model(fhma)
years = 10
df = pd.DataFrame([[1000,200]]*years,index=range(2018,2018+years),columns=['Y','IM'])
df.loc[:,'PA'] = 0.9
df.loc[:,'PC'] = 0.9
start = mhma(df,silent=True)                         
base = mma(start,silent=True)

#md.modeldiff(mma)
#print(md.calculate_allmat(mma,base,2018))
md.get_compagnion(mma,mma.lastdf,2018)

In [ ]:
def geteigen(mul,acc,years=100,show=False):
    '''Function which creates a Samuelson Multiplier accelerator model, runs it and 
    calculates the eigenvalues for the compaignion matrix in order to evaluate stability'''
    
    
    fam   = f'''
    frml <>  y = c+i $ 
    frml <I> c = {mul} * y(-1) $
    frml <>  i = {acc} * (c-c(-1)) + im $'''

    mma = mc.model(fam,modelname = 'Accelerator multiplicator model')

    df = pd.DataFrame([[1000,200]]*years,index=range(2018,2018+years),columns=['Y','IM'])

    start = mc.model(f'c = {mul} * y(-1)')(df,silent=True) # Generate lagged variables for c 
    base  = mma(start,silent=True)     # Solve the model 

    comp = md.get_compagnion(mma,mma.lastdf,2021) # get the compagnion matriox

    compeeig=np.linalg.eig(comp)       # find the eigenvalues 
    if show:
        _ = mma[['Y','C','i']].plot()  # Show the solution
        md.eigplot(compeeig[0])        # show the eigenvalues 
    return  

Stability   

In [ ]:
geteigen(mul=0.5,acc=0,years=30,show=1)

Explosion 

In [ ]:
geteigen(mul=0.9,acc=2,years=30,show=1)

Exploding oscillations

In [ ]:
geteigen(mul=0.6,acc=2,years=30,show=1)

Perpetual oscillations 

In [ ]:
geteigen(mul=0.5,acc=2,years=30,show=1)

Dampened oscillations 

In [ ]:
geteigen(mul=0.7,acc=1,years=30,show=1)